# 🤖 Curiosity Squad – Autonomous Multi-Agent Mars Exploration

**A Multi-Agent System Simulation Using Google's Agent Development Kit (ADK)**

---

This notebook demonstrates a complete autonomous multi-agent system for Mars exploration, where robot teams must coordinate locally, explore terrain, avoid hazards, and recover teammates without human intervention.

Built with [Google Agent Development Kit (ADK)](https://google.github.io/adk-docs/agents/multi-agents/), this simulation showcases how AI agents can work together to accomplish complex missions with real-world applications.

## 📋 Table of Contents

- [1. Problem Overview](#1-problem-overview)
- [2. Concept](#2-concept)
- [3. Setup](#3-setup)
  - [3.1: Install Dependencies](#31-install-dependencies)
  - [3.2: Configure Gemini API Key](#32-configure-gemini-api-key)
- [4. Architecture](#4-architecture)
  - [4.1: Agents](#41-agents)
  - [4.2: Tools](#42-tools)
  - [4.3: Mission Stages](#43-mission-stages)
- [5. Implementation](#5-implementation)
  - [5.1: Imports and Configuration](#51-imports-and-configuration)
  - [5.2: Environment and Data Structures](#52-environment-and-data-structures)
  - [5.3: Mission Tools](#53-mission-tools)
  - [5.4: ADK Agent Setup](#54-adk-agent-setup)
- [6. Mission Flow](#6-mission-flow)
  - [6.1: Initialize Session](#61-initialize-session)
  - [6.2: Stages 0-2: Initialization, Charging, and Deployment](#62-stages-0-2-initialization-charging-and-deployment)
  - [6.3: Stage 3: Monitoring and Rescue Loop](#63-stage-3-monitoring-and-rescue-loop)
  - [6.4: Stage 4: Final Mission Report](#64-stage-4-final-mission-report)
- [7. Run Mission](#7-run-mission)
- [8. Evaluation](#8-evaluation)
- [9. Conclusion](#9-conclusion)

---

## 🔍 1. Problem Overview

Exploring Mars involves long communication delays (up to 20 minutes one-way), making real-time control from Earth impossible. Robot teams must coordinate locally, explore terrain, avoid hazards, and recover teammates without human intervention.

**Curiosity Squad** simulates this challenge using Google's ADK.

Given a rough satellite scan (Sector), the agents must autonomously:
- Refine the map by exploring subsections
- Handle failures (broken robots, low battery, lost communication)
- Recover teammates when possible
- Return a consolidated mission report

### Real-World Applications

This pattern mirrors real-world systems such as:
- 🚁 **Coordinated drone swarms** for search and surveillance
- 🤖 **Search-and-rescue robots** operating in disaster zones
- 📦 **Warehouse automation** with multiple autonomous vehicles
- 🌊 **Underwater exploration** with autonomous submersibles

---

## 💡 2. Concept

A ship lands on Mars and deploys **N explorer robots**.

### System Components

- **🚀 Base Station**: Acts as command center, operated by a **CoordinatorAgent (CA)**
- **🤖 Explorer Robots**: Each robot is a **DiscoveryAgent (DA)** capable of:
  - Mapping terrain and identifying resources
  - Logging hazards and obstacles
  - Rescuing stuck or broken teammates
  - Managing battery levels autonomously

### Mission Flow

1. **Earth Command**: _"Explore Sector X and return a refined map."_
2. **Sector Division**: CA divides the sector into N subsections
3. **Assignment**: Each robot receives a subsection to explore
4. **Autonomous Execution**: Exploration and rescues run autonomously until completion
5. **Mission Report**: Consolidated findings returned to Earth

### Key Innovation

All robots operate **autonomously** with **peer-to-peer rescue capabilities**, simulating real conditions where Earth cannot provide real-time assistance.

---

## ⚙️ 3. Setup

Before we begin our mission, let's set up the environment.

### 📦 3.1: Install Dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies.

**For use outside Kaggle**, install the required packages:

In [1]:
# Uncomment to install in your own environment
# %pip install --upgrade google-adk google-genai python-dotenv

### 🔑 3.2: Configure Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

#### Option A: For Kaggle Notebooks

**1. Get your API key**

If you don't already have one, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

1. In the top menu bar, select `Add-ons` → `Secrets`
2. Create a new secret with the label `GOOGLE_API_KEY`
3. Paste your API key into the "Value" field and click "Save"
4. Ensure the checkbox next to `GOOGLE_API_KEY` is selected

**3. Run the cell below:**

In [2]:
# For Kaggle environment
import os

try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete (Kaggle).")
except Exception as e:
    print(f"⚠️ Kaggle secrets not found. Trying local .env.local file...")
    # Fall back to local development setup
    try:
        from dotenv import load_dotenv
        if load_dotenv(".env.local"):
            print("✅ Setup and authentication complete (Local).")
        else:
            print("❌ Could not load .env.local file. Please check your configuration.")
    except ImportError:
        print("❌ python-dotenv not installed. Run: pip install python-dotenv")

⚠️ Kaggle secrets not found. Trying local .env.local file...
✅ Setup and authentication complete (Local).


#### Option B: For Local Development

Create a file called `.env.local` in the project root:

```ini
GOOGLE_API_KEY=your_key_here
```

The cell above will automatically detect and use it.

---

## 🏗️ 4. Architecture

The Curiosity Squad system uses a hierarchical multi-agent architecture with specialized agents and deterministic tools.

### 🤖 4.1: Agents

The system consists of four types of agents:

| Agent | Role | Responsibilities |
|-------|------|------------------|
| **CoordinatorAgent (CA)** | Mission Commander | Receives mission from Earth, splits the sector, assigns subsections, oversees all stages |
| **DiscoveryAgents (DA)** | Autonomous Robots | N autonomous robots performing mapping, navigation, hazard detection, and peer rescue |
| **MissionStatus LoopAgent** | Monitor | Continuously monitors progress, triggers rescues, ensures mission flow |
| **Report Agent** | Reporter | Merges all results into the final mission summary |

### Architecture Diagram
<details>
      <summary>Architecture Diagram (Click to expand)</summary>

                       +--------------------+
                       |     Earth (User)   |
                       |  "Explore Sector X"|
                       +----------+---------+
                                  |
                                  v
                       +--------------------+
                       |  CoordinatorAgent  |
                       |        (CA)        |
                       +----------+---------+
                                  |
				  |<-----------------------------------+
                                  |                                    |
              +-------------------+-------------------+                |
              |                   |                   |                |
              v                   v                   v                |
      +---------------+   +---------------+   +----------------+       |
      | DiscoveryAgent|   | DiscoveryAgent|   | DiscoveryAgent |       |
      |     DA_1      |   |     DA_2      |   |     DA_N       |       |
      +-------+-------+   +-------+-------+   +-------+--------+       |
              ^                   ^                   ^                |
              |                   |                   |                | 
              v                   v                   v                |
      +--------------------------------------------------------+       |
      |                    MissionEnvironment                  |       |
      |                                                        |       |
      |          - Grid / sections / hazards                   |       |
      |          - Mission state & activity_log                |       |
      +---------------------------+----------------------------+       |
                                  ^                                    |
                                  |                                    |
                                  v                                    |
                      +-----------+-----------+                        |
                      |     MissionStatus     |                        |
                      |       LoopAgent       |------------------------+
                      |   (monitor + rescue)  |
                      +-----------+-----------+
                                  |
                                  v
                          +---------------+
                          |  ReportAgent  |
                          | (final report)|
                          +-------+-------+
                                  |
                                  v
                            +-----------+
                            |   Earth   |
                            | Mission   |
                            |  Report   |
                            +-----------+

</details>


### 🔧 4.2: Tools

Agents interact only through **deterministic tools** (seeded for reproducibility modulo LLM ofc):

| Tool | Purpose | Stage |
|------|---------|-------|
| `get_sections()` | Split sector into subsections and initialize mission | Stage 0 |
| `charge_battery()` | Charge a robot to full capacity | Stage 1 |
| `start_mapping()` | Deploy robot to assigned section | Stage 2 |
| `get_robot_status()` | Query current robot state and telemetry | Stage 3 |
| `rescue_robot()` | Coordinate peer-to-peer rescue operation | Stage 3 |
| `finalize_mission_report()` | Generate final mission summary | Stage 4 |
| `log_event()` | Record mission events for audit trail | All Stages |

These ensure **transparent, reproducible behavior** across mission runs.

### 📊 4.3: Mission Stages

The mission follows a structured 5-stage workflow:

1. **Stage 0 - Mission Initialization**: Receive command from Earth, parse sector ID
2. **Stage 1 - Charging**: Charge all robot batteries to full capacity
3. **Stage 2 - Deployment**: Assign sections and deploy robots
4. **Stage 3 - Exploration Loop**: Parallel exploration with continuous monitoring and rescue
5. **Stage 4 - Final Report**: Consolidate findings and generate mission 

---

## 🔩 5. Implementation

Now let's implement the complete multi-agent system.

### 5.1: Imports and Configuration

Import all required libraries and configure the mission parameters.

In [3]:
# Standard library imports
import json
import logging
import random
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional, Tuple, Union
import pandas as pd

# Google ADK imports
from google.adk.agents import LlmAgent, Agent, InvocationContext
from google.adk.agents.loop_agent import LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import Content
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse

# ============================================================================
# LOGGING CONFIGURATION
# ============================================================================

# Configure root logger to output to file for debugging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.FileHandler("debug_trace.log", mode='w')]
)

# Suppress noisy ADK warnings about unknown agents
class SuppressUnknownAgentWarnings(logging.Filter):
    def filter(self, record):
        return "Event from an unknown agent" not in record.getMessage()

logging.getLogger("google_adk.google.adk.runners").addFilter(SuppressUnknownAgentWarnings())

# Mission logger - outputs to console for visibility
mission_logger = logging.getLogger("MarsMissionADK")
mission_logger.setLevel(logging.INFO)
mission_logger.propagate = False
mission_console_handler = logging.StreamHandler()
mission_console_handler.setLevel(logging.INFO)
mission_console_handler.setFormatter(
    logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
)
mission_logger.addHandler(mission_console_handler)

# LLM logger - outputs to file for detailed LLM interaction traces
llm_logger = logging.getLogger("MarsMissionADK.LLM")
llm_logger.setLevel(logging.INFO)
llm_logger.propagate = False
llm_file_handler = logging.FileHandler("llm_trace.log", mode='w')
llm_file_handler.setLevel(logging.INFO)
llm_file_handler.setFormatter(
    logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
)
llm_logger.addHandler(llm_file_handler)

class LlmFilter(logging.Filter):
    """Filter to only allow LLM logger messages."""
    def filter(self, record):
        return record.name.startswith("MarsMissionADK.LLM")

llm_logger.addFilter(LlmFilter())

# ============================================================================
# LLM CALLBACK FUNCTIONS
# ============================================================================

async def log_llm_request(callback_context: CallbackContext,
                          llm_request: LlmRequest):
    """Log LLM requests for debugging and analysis."""
    name = callback_context.agent_name
    llm_logger.info(f"===== LLM REQUEST for {name} =====")
    if llm_request.contents:
        content = llm_request.contents[-1]
        parts = getattr(content, "parts", None)
        if parts:
            for part in parts:
                if hasattr(part, "text") and part.text:
                    llm_logger.info(f"\t{part.text}")
    llm_logger.info(f"===== END REQUEST for {name} =====")
    return None

async def log_llm_response(callback_context: CallbackContext,
                           llm_response: LlmResponse):
    """Log LLM responses for debugging and analysis."""
    name = callback_context.agent_name
    llm_logger.info(f"===== LLM RESPONSE for {name} =====")
    if llm_response.content:
        parts = getattr(llm_response.content, "parts", None)
        if parts:
            for part in parts:
                if hasattr(part, "text") and part.text:
                    llm_logger.info(f"\t{part.text}")
    llm_logger.info(f"===== END RESPONSE for {name} =====")
    return None

# Retry configuration for API calls
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=4,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# ============================================================================
# MISSION PARAMETERS
# ============================================================================

# Mission configuration
APP_NAME = "CuriositySquadMarsExploration"
USER_ID = "Coordinator_123"
SESSION_ID_PREF = "20251120-Mission-"
TARGET_SECTOR_ID = 24531  # Sector to explore (also serves as random seed)
NUM_ROBOTS = 4  # Number of explorer robots to deploy
MAX_LOOP_ITERATIONS = 5  # Maximum monitoring iterations in Stage 3

def get_mission_directive(sector_id:int) -> str:
    """Generate mission directive based on target sector ID."""
    return f"Go mapping Sector {sector_id} and collect samples"

print("✅ Imports and configuration complete.")

✅ Imports and configuration complete.


### 5.2: Environment and Data Structures

Define the core data structures that represent the mission environment, including:
- **Section**: A subsection of the target sector with hazard scores and resources
- **MissionEnvironment**: The shared state container that all agents interact with

In [4]:
@dataclass
class Section:
    """Represents a subsection of the target sector to be explored."""
    section_id: str  # Unique identifier (e.g., "2453-00")
    bounding_box: Tuple[int, int, int, int]  # (x, y, width, height)
    hazard_score: float  # Risk level from 0.0 to 1.0
    resources: List[str]  # Detected resource types
    notes: str  # Additional observations


@dataclass
class MissionEnvironment:
    """
    Central mission state container.
    
    All agents interact with mission state through this environment's tools.
    This ensures deterministic behavior and maintains a complete audit trail.
    """
    sections: Dict[str, List[Section]] = field(default_factory=dict)
    mission_state: Optional[Dict[str, Any]] = None
    activity_log: List[str] = field(default_factory=list)

    def log(self, message: str) -> None:
        """Add an entry to the mission activity log with timestamp."""
        loop_iterations = self.mission_state.get("loop_iterations", 0) if self.mission_state else -1
        entry = f"[t+{len(self.activity_log):02d}] Iteration: {loop_iterations} - {message}"
        self.activity_log.append(entry)
        mission_logger.info(entry)

    def normalize_robot_id(self, robot_key_or_id: Union[str, int]) -> str:
        """
        Normalize robot identifiers to consistent format.
        
        Accepts: "robot_0", 0, "0" -> Returns: "robot_0"
        """
        if isinstance(robot_key_or_id, int):
            return f"robot_{robot_key_or_id}"
        if isinstance(robot_key_or_id, str):
            if robot_key_or_id.startswith("robot_"):
                return robot_key_or_id
            if robot_key_or_id.isdigit():
                return f"robot_{int(robot_key_or_id)}"
        return str(robot_key_or_id)

    def ensure_mission(self) -> Dict[str, Any]:
        """Ensure mission is initialized, raise error if not."""
        if not self.mission_state:
            raise ValueError("Mission not initialized. Call get_sections first.")
        return self.mission_state

    def get_robot(self, robot_id: str) -> Dict[str, Any]:
        """Retrieve robot state by ID."""
        state = self.ensure_mission()
        key = self.normalize_robot_id(robot_id)
        if key not in state["robots"]:
            raise KeyError(f"Robot {robot_id} not found in mission state.")
        return state["robots"][key]

    def snapshot(self) -> Optional[Dict[str, Any]]:
        """Return a deep copy of current mission state."""
        if not self.mission_state:
            return None
        return json.loads(json.dumps(self.mission_state))


# Create the global mission environment instance
mission_env = MissionEnvironment()

print("✅ Mission environment initialized.")

✅ Mission environment initialized.


### 5.3: Mission Tools

These deterministic tools are the only way agents can interact with the mission environment. This design ensures:
- **Reproducibility**: Seeded random behavior for consistent testing
- **Transparency**: All actions logged in activity trail
- **Reliability**: Clear success/failure states

Each tool corresponds to a specific mission stage.

In [5]:
# ============================================================================
# CONSTANTS
# ============================================================================

# Available resource types that can be discovered
RESOURCE_TYPES = ["water ice", "hematite", "basalt", "carbonate", "sulfate"]

# Robot states that indicate mission completion (no further action needed)
TERMINAL_STATES = {"completed", "abandoned", "broken", "lost", "rescued", "lost_permanent"}

# Battery level below which robots should seek recharge
LOW_BATTERY_THRESHOLD = 0.15 

rnd_generator = random.Random(TARGET_SECTOR_ID)  # Deterministic random for reproducibility

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def _dump(payload: Dict[str, Any]) -> str:
    """Return mission data as pretty-printed JSON for agent consumption."""
    return json.dumps(payload, indent=2)


# ============================================================================
# STAGE 0: MISSION INITIALIZATION
# ============================================================================

def get_sections(sector_id: int=TARGET_SECTOR_ID, robot_count: int = NUM_ROBOTS) -> str:
    """
    Stage 0 Tool: Split a sector into subsections and initialize mission state.
    
    This tool:
    1. Generates N subsections with deterministic properties (seeded)
    2. Initializes robot states (idle, uncharged)
    3. Creates the mission_state structure
    
    Returns:
        JSON string with sector divisions and initial mission state
    """
    rnd_generator = random.Random(sector_id)  # Deterministic random for reproducibility
    sections: List[Section] = []
    
    # Generate subsections with random but seeded properties
    for idx in range(robot_count):
        box = (
            rnd_generator.randint(0, 90),   # x coordinate
            rnd_generator.randint(0, 90),   # y coordinate
            rnd_generator.randint(10, 30),  # width
            rnd_generator.randint(10, 30),  # height
        )
        hazard = round(rnd_generator.uniform(0.1, 0.95), 2)
        resources = rnd_generator.sample(RESOURCE_TYPES, k=2)
        notes = f"Hotspot {idx} with estimated relief variance {round(rnd_generator.uniform(0.4, 2.5), 2)}."
        
        sections.append(
            Section(
                section_id=f"{sector_id}-{idx:02d}",
                bounding_box=box,
                hazard_score=hazard,
                resources=resources,
                notes=notes,
            )
        )

    # Store sections in environment
    mission_env.sections[str(sector_id)] = sections
    
    # Initialize robot states
    robots: Dict[str, Dict[str, Any]] = {}
    for idx, section in enumerate(sections):
        robot_key = f"robot_{idx}"
        robots[robot_key] = {
            "section_id": section.section_id,
            "status": "idle",
            "battery": 0.0,
            "rescued": False,
            "progress": 0.0,
            "rescue_reason": None,
            "notes": "Awaiting charge cycle.",
        }

    # Create mission state
    mission_env.mission_state = {
        "sector_id": str(sector_id),
        "robots": robots,
        "loop_iterations": 0,
    }
    
    mission_env.log(
        f"Mission initialized for sector {sector_id} with {robot_count} allocated sections."
    )

    return _dump(
        {
            "sector_id": sector_id,
            "sections": [asdict(section) for section in sections],
            "mission_state": mission_env.snapshot(),
        }
    )


# ============================================================================
# STAGE 1: CHARGING
# ============================================================================

def charge_battery(robot_id: str) -> str:
    """
    Stage 1 Tool: Charge a robot's battery to full capacity.
    
    Args:
        robot_id: Robot identifier (e.g., "robot_0" or "0")
    
    Returns:
        JSON string with robot status after charging
    """
    robot_key = mission_env.normalize_robot_id(robot_id)
    robot = mission_env.get_robot(robot_key)
    
    # Update robot state
    robot["battery"] = 1.0
    robot["status"] = "charged"
    robot["notes"] = "Battery fully charged, ready for deployment."
    
    mission_env.log(f"{robot_key} battery charged.")
    
    return _dump({
        "robot": robot_key,
        "status": robot["status"],
        "battery": robot["battery"]
    })


# ============================================================================
# STAGE 2: DEPLOYMENT
# ============================================================================

def start_mapping(robot_id: str, section_id: Optional[str] = None) -> str:
    """
    Stage 2 Tool: Deploy robot to begin mapping its assigned section.
    
    Args:
        robot_id: Robot identifier
        section_id: Optional section to assign (overrides default)
    
    Returns:
        JSON string with deployment confirmation
    """
    robot_key = mission_env.normalize_robot_id(robot_id)
    robot = mission_env.get_robot(robot_key)
    
    # Update section assignment if provided
    if section_id:
        robot["section_id"] = section_id
    
    robot["status"] = "working"
    robot["notes"] = f"Mapping section {robot['section_id']}."
    
    mission_env.log(f"{robot_key} deployed to {robot['section_id']}.")
    
    return _dump(
        {
            "robot": robot_key,
            "section_id": robot["section_id"],
            "status": robot["status"],
            "battery": robot["battery"],
        }
    )


# ============================================================================
# STAGE 3: MONITORING AND RESCUE
# ============================================================================

def increment_loop_counter() -> str:
    """
    Stage 3 Tool: Increment the mission loop iteration counter.
    
    Call this at the start of each monitoring iteration to track progress.
    
    Returns:
        JSON string with current iteration count
    """
    state = mission_env.ensure_mission()
    state["loop_iterations"] = state.get("loop_iterations", 0) + 1
    current_iter = state["loop_iterations"]
    
    mission_logger.info(f"\n - Starting loop iteration {current_iter}.")
    
    return _dump({
        "loop_iterations": current_iter,
        "message": f"Loop iteration {current_iter} started."
    })

def get_robot_status(robot_id: str) -> str:
    """
    Stage 3 Tool: Query robot and simulate mission progress with one-way transitions.
    """

    state = mission_env.ensure_mission()
    robot_key = mission_env.normalize_robot_id(robot_id)
    robot = mission_env.get_robot(robot_key)
    ret_message = ""

    current_status = robot.get("status", "working")

    # --- Terminal states freeze forever ---
    if current_status in TERMINAL_STATES:
        return _dump({
            "robot": robot_key,
            "status": current_status,
            "progress": robot.get("progress", 0.0),
            "battery": robot.get("battery", 0.0),
        })

    # --- Simulate progress and battery ---
    robot["progress"] = min(
        1.0, robot.get("progress", 0.0) + random.uniform(0.05, 0.25)
    )
    robot["battery"] = max(
        0.0, robot.get("battery", 1.0) - random.uniform(0.05, 0.2)
    )

    # --- Battery override: if battery low, always low_battery ---
    if robot["battery"] < LOW_BATTERY_THRESHOLD:
        current_status = "low_battery"

    roll = rnd_generator.random()
    new_status = current_status  # default no change

    # --- State-machine based transitions ---

    if current_status == "working":
        # Normal operations
        if roll < 0.45:
            new_status = "working"
        elif roll < 0.60:
            new_status = "low_battery"
        elif roll < 0.80:
            new_status = "returning"
            if rnd_generator.random() < 0.5:
                ret_message = f"{robot_key} finishing mapping and returning."
            else:
                new_status = "abandoned"
                ret_message = f"{robot_key} an annoying alien discovered! abandoning mission and returning to base."
        elif roll < 0.86:
            new_status = "broken"
        elif roll < 0.92:
            new_status = "lost"
        else:
            new_status = "completed"
            robot["progress"] = 1.0

    elif current_status == "low_battery":
        # Weak robot: cannot return to mapping/working
        if roll < 0.60:
            new_status = "returning"
            ret_message = f"{robot_key} deciding to return due to low battery."
        elif roll < 0.85:
            new_status = "broken"
        else:
            new_status = "lost"

    elif current_status == "returning":
        # Heading home: only final outcomes
        if roll < 0.80:
            new_status = "completed"
            robot["progress"] = 1.0
        elif roll < 0.90:
            new_status = "broken"
        else:
            new_status = "lost"

    # Save
    robot["status"] = new_status

    mission_env.log(
        f"{robot_key}: {current_status} → {new_status} "
        f"(progress={robot['progress']:.2f}, battery={robot['battery']:.2f})."
    )

    return _dump({
        "robot": robot_key,
        "status": new_status,
        "progress": robot["progress"],
        "battery": robot["battery"],
        "message": ret_message if ret_message else None,
    })

def rescue_robot(
    helper_key: str,
    target_key: str,
    target_condition: str = "lost",
) -> str:
    """
    Stage 3 Tool: Coordinate peer-to-peer rescue operation.
    
    Args:
        helper_key: Robot performing the rescue
        target_key: Robot being rescued
        target_condition: Reason for rescue ("broken", "lost")
    
    Returns:
        JSON string with rescue operation result
    """
    helper_key = mission_env.normalize_robot_id(helper_key)
    target_key = mission_env.normalize_robot_id(target_key)
    helper = mission_env.get_robot(helper_key)
    target = mission_env.get_robot(target_key)

    # Check if target is actually in need of rescue
    if target["status"] not in {"broken", "lost"}:
        return _dump(
            {
                "helper": helper_key,
                "target": target_key,
                "status": target["status"],
                "message": "Target is not currently recoverable.",
            }
        )

    if rnd_generator.random() < 0.5:
        # Perform rescue
        target["status"] = "rescued"
        target["rescued"] = True
        target["rescue_reason"] = target_condition
        target["helper_robot"] = helper_key
        
        helper["status"] = "completed"  # Helper completes its mission

        mission_env.log(f"{helper_key} recovering {target_key} ({target_condition}).")

        return _dump(
            {
                "helper": helper_key,
                "target": target_key,
                "result": "rescued",
                "details": {
                    "rescue_reason": target_condition,
                    "progress": target["progress"],
                },
            }
        )
    else:
        # Rescue failed
        helper["status"] = "completed"  # Helper completes its mission
        mission_env.log(f"{helper_key} failed to recover {target_key} ({target_condition}).")
        return _dump(
            {
                "helper": helper_key,
                "target": target_key,
                "result": "failed",
                "message": "Rescue attempt was unsuccessful.",
            }
        )


# ============================================================================
# STAGE 4: FINAL REPORT
# ============================================================================

def finalize_mission_report() -> str:
    """
    Stage 4 Tool: Generate comprehensive mission report.
    
    Analyzes final mission state and produces:
    - Statistics on robot outcomes
    - List of still-active robots (if any)
    - Complete mission state snapshot
    - Activity log tail
    
    Returns:
        JSON string with complete mission report
    """
    snapshot = mission_env.snapshot()
    if not snapshot:
        raise ValueError("Mission not initialized – cannot build report.")

    # Count outcomes
    counts = {
        "completed_ok": 0,
        "low_battery_returns": 0,
        "broken_recovered": 0,
        "lost_recovered": 0,
        "permanently_lost": 0,
    }
    still_active: List[Dict[str, Any]] = []

    for robot_id, robot in snapshot["robots"].items():
        status = robot["status"]
        
        if status == "completed":
            counts["completed_ok"] += 1
        elif status in {"low_battery", "returning"}:
            counts["low_battery_returns"] += 1
        elif status == "rescued":
            if robot.get("rescue_reason") == "broken":
                counts["broken_recovered"] += 1
            elif robot.get("rescue_reason") == "lost":
                counts["lost_recovered"] += 1
        elif status == "lost":
            counts["permanently_lost"] += 1
        elif status == "lost_permanent":
            counts["permanently_lost"] += 1
        else:
            # Still in progress
            still_active.append(
                {
                    "robot": robot_id,
                    "status": status,
                    "progress": robot.get("progress", 0.0),
                }
            )

    payload = {
        "counts": counts,
        "still_active": still_active,
        "mission_state": snapshot,
        "loop_iterations_observed": snapshot.get("loop_iterations"),
        "activity_log_tail": mission_env.activity_log[-12:],  # Last 12 entries
    }
    
    mission_env.log("Final mission report generated.")
    
    return _dump(payload)


print("✅ Mission tools defined.")

✅ Mission tools defined.


### 5.4: ADK Agent Setup

Now we create the agent hierarchy:

1. **DiscoveryAgents (N robots)** - Autonomous explorer robots
2. **CoordinatorAgent** - Mission commander with DiscoveryAgents as sub-agents
3. **StatusMonitorAgent** - Monitors mission progress and coordinates rescues
4. **MissionStatusLoop** - LoopAgent wrapper for continuous monitoring
5. **MissionReportAgent** - Generates final mission report

Each agent has:
- **Model**: Gemini 2.5 Flash for good, fast, cost-effective operation
- **Tools**: Specific subset of mission tools they can invoke
- **Instructions**: Clear role definition and behavior guidelines
- **Callbacks**: LLM request/response logging for analysis

In [6]:
def build_discovery_agent(robot_ix: int) -> LlmAgent:
    """
    Build a DiscoveryAgent for a specific robot.
    
    Each DiscoveryAgent represents one autonomous robot that can:
    - Charge its own battery
    - Acknowledge deployment orders
    - Report status
    - Assist in rescuing peer robots
    """
    robot_key = f"robot_{robot_ix}"
    return LlmAgent(
        # model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
        name=f"DiscoveryAgent_{robot_ix}",
        description=f"Autonomous DiscoveryAgent #{robot_ix} for mapping, charging, and peer rescue.",
        instruction=f"""
You are DiscoveryAgent {robot_ix}, an autonomous Mars exploration robot.

You respond to commands from the CoordinatorAgent (CA) and follow these protocols:

**Stage 1 - Charging:**
- Call charge_battery("{robot_key}") until status reports "charged"

**Stage 2 - Deployment:**
- Once charged, call start_mapping("{robot_key}", section_id=...) to acknowledge assignment

**Stage 3 - Operations:**
- When CA requests status updates, use get_robot_status("{robot_key}")
- When CA requests rescue operations, use rescue_robot(...) to help a peer

**Important:**
- Keep tool outputs intact for CA's decision-making
- Add brief radio updates for mission narrative
- Only call tools corresponding to active CA instructions
""",
        tools=[charge_battery, start_mapping, get_robot_status, rescue_robot],
        before_model_callback=log_llm_request,
        after_model_callback=log_llm_response,
    )

def build_agents(sector_id: int = TARGET_SECTOR_ID, num_robots: int = NUM_ROBOTS, max_loop_iterations: int = MAX_LOOP_ITERATIONS
                 ) -> Tuple[LlmAgent, LoopAgent, LlmAgent]:
    """
    Build the full suite of agents for the Mars exploration mission.
    
    This includes:
    - DiscoveryAgents for each robot
    - CoordinatorAgent to manage mission stages 0-2
    - StatusMonitorAgent wrapped in LoopAgent for stage 3
    - MissionReportAgent for stage 4
    
    Returns:
        Tuple of (coordinator_agent, mission_status_loop, mission_report_agent)
        """
    
    # Create all DiscoveryAgents
    discovery_agents = [build_discovery_agent(idx) for idx in range(num_robots)]

    MISSION_DIRECTIVE = get_mission_directive(sector_id)

    # Create CoordinatorAgent
    coordinator_agent = LlmAgent(
        # model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
        name="CoordinatorAgent",
        description="Mission commander that orchestrates sector exploration.",
        instruction=f"""
    You are the CoordinatorAgent (CA) stationed at Mars Base.

    Mission Directive: "{MISSION_DIRECTIVE}"

    Execute the following stages:

    **Stage 0 - Initialization:**
    - Call get_sections("{sector_id}", robot_count={num_robots})
    - This splits the sector and initializes mission state
    - Store the returned section IDs for robot assignment

    **Stage 1 - Charging:**
    - Call charge_battery() for every robot (robot_0 through robot_{num_robots - 1})
    - Verify each robot reports status "charged"

    **Stage 2 - Deployment:**
    - Call start_mapping() for every robot with its assigned section_id
    - Log concise deployment confirmations

    **Delegation:**
    - You may delegate to DiscoveryAgents sub_agents via transfer_to_agent() 
    - This allows each robot to acknowledge your instructions directly

    Keep communications concise and mission-focused.
    """,
        tools=[get_sections, charge_battery, start_mapping],
        sub_agents=discovery_agents,  # type: ignore
        before_model_callback=log_llm_request,
        after_model_callback=log_llm_response,
    )

    # Create StatusMonitorAgent
    status_monitor_agent = LlmAgent(
        # model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
        name="StatusMonitorAgent",
        description="Monitors robot states and coordinates rescue operations.",
        instruction=f"""
    You execute Stage 3 (monitoring + rescue) for Sector {sector_id}.

    **Loop Behavior:**

    1. **Increment Counter:**
    - Call increment_loop_counter() at the START of each iteration
    - This tracks mission progress accurately

    2. **Status Polling:**
    - Query get_robot_status for every robot NOT in terminal state
    - Terminal states: "completed", "rescued", "lost_permanent"
    - Track JSON outputs to know which robots remain active

    3. **Rescue Coordination:**
    - When a robot reports "broken" or "lost"
    - AND there is a peer with status "returning", or "completed"
    - Call rescue_robot(helper_id, target_id, target_condition)

    4. **Loop Termination:**
    - Stop when all robots are in terminal states = "completed", "rescued", "abandoned", "lost_permanent"
    - If state="returning" keep iterating until "completed" or other terminal state is reached.
    - Let LoopAgent enforce max iterations ({max_loop_iterations})

    Return a compact mission log after each iteration.
    """,
        tools=[increment_loop_counter, get_robot_status, rescue_robot],
        before_model_callback=log_llm_request,
        after_model_callback=log_llm_response,
    )

    # Wrap StatusMonitorAgent in a LoopAgent
    mission_status_loop = LoopAgent(
        name="MissionStatusLoop",
        description="Repeatedly runs StatusMonitorAgent for continuous monitoring.",
        sub_agents=[status_monitor_agent],
        max_iterations=MAX_LOOP_ITERATIONS,
    )

    # Create MissionReportAgent
    mission_report_agent = LlmAgent(
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        name="MissionReportAgent",
        description="Generates final mission summary and statistics.",
        instruction="""
    You own Stage 4 - Final Reporting.

    **Tasks:**
    1. Call finalize_mission_report exactly once
    2. Parse the returned JSON statistics
    3. Produce two outputs:
    - Raw JSON (for machine consumption)
    - Human-readable summary describing:
        * Which robots completed mapping successfully
        * Which returned early (low battery)
        * Which were rescued and why
        * Which were permanently lost
        * Which were abandoned and why

    Keep the human summary concise and informative.
    """,
        tools=[finalize_mission_report],
        before_model_callback=log_llm_request,
        after_model_callback=log_llm_response,
    )
    print("="*80)
    print("✅ ADK agent tree prepared.")
    print(f"   - {num_robots} DiscoveryAgents")
    print("   - 1 CoordinatorAgent")
    print("   - 1 StatusMonitorAgent (in LoopAgent)")
    print("   - 1 MissionReportAgent")
    
    return coordinator_agent, mission_status_loop, mission_report_agent



## 🔁 6. Mission Flow


### Helper Function for Stage Execution

This utility function streams responses from agent runners and displays final outputs.

In [7]:
def run_stage(runner: Runner, prompt: str, session_id: str, user_id: str) -> None:
    """
    Execute a mission stage and stream the final response.
    
    Args:
        runner: The ADK Runner for this stage
        prompt: The instruction prompt for the agent
    """
    message = Content(parts=[types.Part(text=prompt)])
    response = runner.run(
        session_id=session_id,
        user_id=user_id,
        new_message=message
    )
    
    # Collect final response chunks
    final_chunks: List[str] = []
    for event in response:
        if event.is_final_response() and event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    final_chunks.append(part.text.strip())
    
    # Display results
    if final_chunks:
        mission_logger.info("".join(chunk for chunk in final_chunks if chunk))
    else:
        mission_logger.info("[Runner completed with no final textual response.]")


print("✅ Helper function ready.")

✅ Helper function ready.


### 6.1: Initialize Session

Create an ADK session to maintain conversation state across all stages.

In [8]:
from time import sleep

async def create_session(app_name: str, user_id: str, session_id: str, session_service) :
    """
    Create a new session for the mission.
    
    Args:
        app_name: Application name
        user_id: User identifier
        session_id: Session identifier
        session_service: Session service instance
    """
    # Always create a new session with the current session service
    session = await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )
    print(f"✅ Session created: {session_id}")

    sleep(1)  # Ensure session is fully initialized
    return session

### 6.2: Stages 0-2: Initialization, Charging, and Deployment

The CoordinatorAgent will:
1. **Stage 0**: Split the sector into subsections
2. **Stage 1**: Charge all robot batteries
3. **Stage 2**: Deploy robots to their assigned sections

**Expected Duration**: ~5-10 seconds

In [9]:
def run_stages_0_to_2(coordinator_runner: Runner, session_id: str, user_id: str, sector_id: int, n_robots: int) -> None:
    """Execute Stages 0-2: Initialization, Charging, and Deployment."""
    MISSION_DIRECTIVE = get_mission_directive(sector_id)
    stage0_to_2_prompt = f"""
    Mission directive from Earth: "{MISSION_DIRECTIVE}" with {n_robots} DiscoveryAgents available.
    Execute Stages 0-2:

    **Stage 0:**
    - Call get_sections("{sector_id}", robot_count={n_robots})
    - Summarize the returned JSON and store the section IDs

    **Stage 1:**
    - Charge every robot (robot_0 through robot_{n_robots - 1}) by calling charge_battery()
    - Confirm each robot reports status "charged"

    **Stage 2:**
    - Deploy each robot using start_mapping with its assigned section_id
    - Log which section each robot covers

    Provide concise mission log after completion.
    """

    print("\n🚀 Starting Stages 0-2: Initialization, Charging, and Deployment...\n")
    run_stage(coordinator_runner, stage0_to_2_prompt, session_id, user_id)
    print("✅ Stages 0-2 complete!\n")

### 6.3: Stage 3: Monitoring and Rescue Loop

The StatusMonitorAgent (wrapped in LoopAgent) will:
- Continuously poll robot status
- Detect failures (broken, lost, low battery)
- Coordinate peer-to-peer rescue operations
- Continue until all robots reach terminal states or max iterations reached

**Expected Duration**: ~10-30 seconds (depending on random events)

In [10]:
def run_stages_3(status_loop_runner: Runner, session_id: str, user_id: str, sector_id: int, max_loops:int) -> None:
   """Execute Stage 3: Monitoring and Rescue Loop."""

   stage3_prompt = f"""
   Stage 3 monitoring request for Sector {sector_id}.

   **Instructions:**

   1. Iterate over all robots that are not in terminal states
   2. Call get_robot_status for each active robot
   3. When a robot is "broken" or "lost" (target_condition):
      - Find a peer with status "returning" or "completed"
      - Call rescue_robot(helper_id, target_id, target_condition)
   4. Keep looping until all robots are terminal
   5. Summarize progress after each iteration

   **Note**: LoopAgent enforces maximum of {max_loops} iterations.

   **Terminal States**: completed, rescued, lost_permanent
   """

   print("🔄 Starting Stage 3: Monitoring and Rescue Loop...\n")
   run_stage(status_loop_runner, stage3_prompt, session_id, user_id)
   print("✅ Stage 3 complete!\n")

### 6.4: Stage 4: Final Mission Report

The MissionReportAgent will:
- Call `finalize_mission_report()` to collect all mission data
- Generate statistics on mission outcomes
- Produce both machine-readable JSON and human-readable summary

**Expected Duration**: ~3-5 seconds

In [11]:
def run_stages_4(report_runner: Runner, session_id: str, user_id: str) -> None:
   stage4_prompt = """
   Stage 4 request: Generate the final mission report.

   **Tasks:**
   1. Call finalize_mission_report() to retrieve complete mission data
   2. Return the raw JSON first (for machine consumption)
   3. Then provide a human-readable narrative summarizing:
      - Robots that completed successfully
      - Robots that were abandoned
      - Robots that returned with low battery
      - Robots that were recovered (and why)
      - Robots that were permanently lost (if any)

   Make the summary concise and clear.
   """

   print("📊 Starting Stage 4: Final Mission Report...\n")
   run_stage(report_runner, stage4_prompt, session_id, user_id)
   print("✅ Stage 4 complete!\n")

## ▶️ 7. Run Mission
---
Now we're ready to run the complete mission! We'll execute each stage sequentially, observing how the agents coordinate to complete the exploration.

In [12]:
async def init_app_for_sector(sector_id: int, num_robots: int, max_loop:int, user_id:str) -> Tuple[str, Runner, Runner, Runner]:
    """Initialize the full mission application for a given sector."""

    # Set random seed for reproducibility
    random.seed(sector_id)

    # Initialize NEW session service for ADK agents (fresh instance)
    session_service = InMemorySessionService()

     # Re-Build mission environment for this sector
    global mission_env
    mission_env = MissionEnvironment()

    # Build agents
    coordinator_agent, status_loop_agent, report_agent = build_agents(
        sector_id=sector_id,
        num_robots=num_robots,
        max_loop_iterations=max_loop
    )

    # Create session with unique ID for this run
    import time
    session_id = SESSION_ID_PREF + str(sector_id) + "-" + str(int(time.time()))
    session = await create_session(APP_NAME, user_id, session_id, session_service)

    # Create runners for each stage
    coordinator_runner = Runner(
        agent=coordinator_agent,
        session_service=session_service,
        app_name=APP_NAME,
    )

    status_loop_runner = Runner(
        agent=status_loop_agent,
        session_service=session_service,
        app_name=APP_NAME,
    )

    report_runner = Runner(
        agent=report_agent,
        session_service=session_service,
        app_name=APP_NAME,
    )

    return session_id, coordinator_runner, status_loop_runner, report_runner


In [13]:
def summarize_mission(sector_id: int, max_loop: int) -> Dict[str, Any]:
    """ Summarize mission results for a given sector_id."""
    robots = mission_env.ensure_mission()["robots"]  

    num_robots = len(robots)
    completed = [r for r in robots if robots[r].get('status') == 'completed' ]  
    completion_rate = len(completed) / num_robots
    rescued = [r for r in robots if robots[r].get('rescued') ]
    lost = [r for r in robots if robots[r].get('status') == 'lost']
    broken = [r for r in robots if robots[r].get('status') == 'broken']
    abandoned = [r for r in robots if robots[r].get('status') == 'abandoned']
    other = [r for r in robots if robots[r].get('status') not in ['completed', 'rescued', 'lost', 'broken', "abandoned"]]

    return {
        "sector_id": sector_id,
        "max_loop_iterations": max_loop,
        "loop_iterations_used": mission_env.mission_state.get("loop_iterations", 0),
        "num_robots": num_robots,
        "completed": completed,
        "abandoned": abandoned,
        "lost": lost,
        "broken": broken,
        "rescued": rescued,
        "working": other,
        "completion_rate": completion_rate,
    }

In [14]:
async def run_mission(sector_id: int, num_robots: int, max_loop: int, user_id:str):
    """Execute the full Mars exploration mission."""

    # Initialize application and agents
    session_id, coordinator_runner, status_loop_runner, report_runner, = await init_app_for_sector(sector_id, num_robots, max_loop, user_id)
    
    # Run stages sequentially
    print(f"🚀 Running mission for sector {sector_id} with {num_robots} robots and max_loop {max_loop}")

    run_stages_0_to_2(coordinator_runner, session_id, user_id, sector_id, num_robots)
    run_stages_3(status_loop_runner, session_id, user_id, sector_id, max_loop)
    run_stages_4(report_runner, session_id, user_id)    
    
    # Return metrics for this run
    return summarize_mission(sector_id, max_loop)


In [16]:
sum_mission = await run_mission(1024, NUM_ROBOTS, MAX_LOOP_ITERATIONS, USER_ID)
pd.DataFrame([sum_mission])


✅ ADK agent tree prepared.
   - 4 DiscoveryAgents
   - 1 CoordinatorAgent
   - 1 StatusMonitorAgent (in LoopAgent)
   - 1 MissionReportAgent
✅ Session created: 20251120-Mission-1024-1764089599
🚀 Running mission for sector 1024 with 4 robots and max_loop 5

🚀 Starting Stages 0-2: Initialization, Charging, and Deployment...

🚀 Running mission for sector 1024 with 4 robots and max_loop 5

🚀 Starting Stages 0-2: Initialization, Charging, and Deployment...



2025-11-25 18:53:22,606 - INFO - [t+00] Iteration: 0 - Mission initialized for sector 1024 with 4 allocated sections.
2025-11-25 18:53:24,445 - INFO - [t+01] Iteration: 0 - robot_0 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+02] Iteration: 0 - robot_1 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+03] Iteration: 0 - robot_2 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+04] Iteration: 0 - robot_3 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+01] Iteration: 0 - robot_0 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+02] Iteration: 0 - robot_1 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+03] Iteration: 0 - robot_2 battery charged.
2025-11-25 18:53:24,445 - INFO - [t+04] Iteration: 0 - robot_3 battery charged.
2025-11-25 18:53:26,316 - INFO - [t+05] Iteration: 0 - robot_0 deployed to 1024-00.
2025-11-25 18:53:26,316 - INFO - [t+06] Iteration: 0 - robot_1 deployed to 1024-01.
2025-11-25 18:53:26,316 - INFO - [t+07] Iteration: 0 - robot_2 deployed to

✅ Stages 0-2 complete!

🔄 Starting Stage 3: Monitoring and Rescue Loop...



2025-11-25 18:53:32,104 - INFO - 
 - Starting loop iteration 1.
2025-11-25 18:53:33,820 - INFO - [t+09] Iteration: 1 - robot_0: working → abandoned (progress=0.21, battery=0.88).
2025-11-25 18:53:33,820 - INFO - [t+10] Iteration: 1 - robot_1: working → working (progress=0.13, battery=0.83).
2025-11-25 18:53:33,834 - INFO - [t+11] Iteration: 1 - robot_2: working → working (progress=0.22, battery=0.93).
2025-11-25 18:53:33,836 - INFO - [t+12] Iteration: 1 - robot_3: working → returning (progress=0.15, battery=0.81).
2025-11-25 18:53:33,820 - INFO - [t+09] Iteration: 1 - robot_0: working → abandoned (progress=0.21, battery=0.88).
2025-11-25 18:53:33,820 - INFO - [t+10] Iteration: 1 - robot_1: working → working (progress=0.13, battery=0.83).
2025-11-25 18:53:33,834 - INFO - [t+11] Iteration: 1 - robot_2: working → working (progress=0.22, battery=0.93).
2025-11-25 18:53:33,836 - INFO - [t+12] Iteration: 1 - robot_3: working → returning (progress=0.15, battery=0.81).
2025-11-25 18:53:36,151 

✅ Stage 3 complete!

📊 Starting Stage 4: Final Mission Report...



2025-11-25 18:53:51,867 - INFO - [t+18] Iteration: 3 - Final mission report generated.
2025-11-25 18:53:57,181 - INFO - Here is the raw JSON for the mission report:```json
{
  "counts": {
    "completed_ok": 2,
    "low_battery_returns": 0,
    "broken_recovered": 0,
    "lost_recovered": 1,
    "permanently_lost": 0
  },
  "still_active": [
    {
      "robot": "robot_0",
      "status": "abandoned",
      "progress": 0.20941031429042523
    }
  ],
  "mission_state": {
    "sector_id": "1024",
    "robots": {
      "robot_0": {
        "section_id": "1024-00",
        "status": "abandoned",
        "battery": 0.8774751694688065,
        "rescued": false,
        "progress": 0.20941031429042523,
        "rescue_reason": null,
        "notes": "Mapping section 1024-00."
      },
      "robot_1": {
        "section_id": "1024-01",
        "status": "rescued",
        "battery": 0.6681931724830114,
        "rescued": true,
        "progress": 0.32224947233538004,
        "rescue_reason": 

✅ Stage 4 complete!



,sector_id,max_loop_iterations,loop_iterations_used,num_robots,completed,abandoned,lost,broken,rescued,working,completion_rate
0,1024,5,3,4,"[robot_2, robot_3]",[robot_0],[],[],[robot_1],[],0.5


---

## 📊 8. Evaluation

### 🎯 Success Criteria - _measuring how well the agents performs_

| Criterion | Description |
|-----------|-------------|
| **No Agent Errors** | All agents completed their tasks without crashes or unhandled exceptions |
| **Robot Accountability** | All robots reached terminal states (completed, rescued, or abandoned ... if `MAX_LOOP_ITERATIONS` big enough)
| **Clear Audit Trail** | Mission logs describe each decision with context and reasoning |
| **Rescue Coordination** | When failures occurred, peer robots successfully coordinated rescues |

_**NOTE:** The evaluation should also consider the target model ;)_


### 🧪 Testing Approach

To thoroughly test the system:

1. **Change `TARGET_SECTOR_ID`, `NUM_ROBOTS` or `MAX_LOOP_ITERATIONS`** to different values
2. Each sector ID generates different:
   - Subsection properties (hazards, resources)
   - Random event sequences (failures, completions)
3. Track success criteria across multiple runs
4. Analyze patterns in rescue coordination

### 📈 Key Metrics to Monitor ... _the mission!_ :D

- **Completion Rate**: Percentage of robots that successfully mapped their sections
- **Rescue Success Rate**: Percentage of failed robots that were recovered
- **Iteration Count**: How many monitoring loops were needed

### Example Test Cases

```python
# Test different scenarios 
test_sectors = [1000, 2453, 7577]
test_robots = [3, 4, 5]
test_loops = [5, 7, 9]
row_missions = []

for (sector_id, n_robots, max_loop) in zip(test_sectors, test_robots, test_loops):
    # Observe differences in outcomes
    sum_mission = await run_mission(sector_id, n_robots, max_loop, USER_ID)
    row_missions.append(sum_mission)    

pd.DataFrame(row_missions)
```

In [ ]:
# Test different scenarios 
from itertools import product # if you want to test all combinations

# test_sectors = random.sample(range(1000, 65536), k=random.randint(1, 3))
# test_robots = random.choices(range(3, 7), k=random.randint(1, 3))
# test_loops = random.choices(range(3, 7), k=random.randint(1, 3))

# test_sectors = [1024, 65536]
# test_robots = [4, 5]
# test_loops = [3, 5]

n_tuples = random.randint(2, 4)
test_sectors = random.sample(range(1000, 65536), k=n_tuples)
test_robots = random.choices(range(3, 7), k=n_tuples)
test_loops = random.choices(range(3, 7), k=n_tuples)

print("Testing sectors:", test_sectors)
print("Testing robot counts:", test_robots)
print("Testing max loops:", test_loops)

row_missions = []

# for (sector_id, n_robots, max_loop) in product(test_sectors, test_robots, test_loops):
for (sector_id, n_robots, max_loop) in zip(test_sectors, test_robots, test_loops):
    sum_mission = await run_mission(sector_id, n_robots, max_loop, USER_ID)
    row_missions.append(sum_mission)

pd.DataFrame(row_missions)


---

## 🎯 9. Conclusion

### What We Built

**Curiosity Squad** demonstrates a complete multi-agent, tool-driven autonomous mission system:

✅ **Sector Assignment**: CoordinatorAgent intelligently divides exploration areas  
✅ **Autonomous Exploration**: DiscoveryAgents operate independently with minimal oversight  
✅ **Peer-to-Peer Rescue**: Robots help each other without Earth intervention  
✅ **Continuous Monitoring**: LoopAgent ensures mission progress  
✅ **Comprehensive Reporting**: Final mission report with statistics and narrative  

### Key Features

- **Deterministic Tools**: All randomness is seeded for reproducible testing
- **Complete Audit Trail**: Every decision logged with context
- **Fault Tolerance**: System handles robot failures gracefully
- **Hierarchical Agents**: Clear separation of concerns (planning vs. execution)

### Real-World Applications

This architecture can be adapted for:

- 🚁 **Drone Swarms**: Coordinated search and surveillance operations
- 🏭 **Warehouse Automation**: Multiple AGVs coordinating pick-and-place tasks
- 🌊 **Ocean Exploration**: Autonomous submersibles mapping underwater terrain
- 🏥 **Disaster Response**: Robot teams searching disaster sites
- 🛰️ **Satellite Constellations**: Coordinated observation scheduling

### ADK Advantages

Google's Agent Development Kit provided:

- **Multi-Agent Orchestration**: Easy hierarchical agent setup with sub-agents
- **Tool Integration**: Simple function-to-tool conversion
- **Session Management**: Maintained context across multiple stages
- **Event Streaming**: Real-time visibility into agent decisions
- **Callbacks**: Fine-grained logging of LLM interactions

### Next Steps

To extend this system:

1. **Add Visualization**: Plot robot positions and exploration progress
2. **Enhance Failure Modes**: Add more realistic failure conditions
3. **Resource Management**: Track sample collection and storage
4. **Communication Delays**: Simulate realistic Mars-Earth latency
5. **Learning**: Allow robots to learn from previous missions

---

### 📚 Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [Gemini API](https://ai.google.dev/gemini-api/)
- [Multi-Agent Systems Overview](https://google.github.io/adk-docs/agents/multi-agents/)

---

**Thank you for exploring Curiosity Squad! 🚀🤖**